In [1]:
import numpy as np
import pandas as pd
from datetime import date


In [2]:
# local connection information
import local_db
connection = local_db.connection()


In [3]:
today = date.today()
today_str = today.strftime('%Y%m%d')
print(today_str)

20180522


In [4]:
sections_begin_year = '2011'


In [5]:
sql_str = "SELECT * FROM SECTIONSCHEDULE WHERE " + \
          f"ACADEMIC_YEAR >= '{sections_begin_year}' " + \
          "AND ACADEMIC_TERM IN ('FALL', 'SPRING', 'SUMMER') " + \
          "AND ACADEMIC_SESSION IN ('MAIN', 'CULN', 'EXT', 'FNRR', 'HEOP', " + \
          " 'SLAB', 'BLOCK A', 'BLOCK AB', 'BLOCK B') "
df_sections = pd.read_sql_query(sql_str, connection)


In [6]:
df = df_sections[['ACADEMIC_YEAR', 'ACADEMIC_TERM', 'ACADEMIC_SESSION',
                  'EVENT_ID', 'EVENT_SUB_TYPE',
                  'SECTION', 'DAY',
                  'START_TIME', 'END_TIME',
                  'BUILDING_CODE', 'ROOM_ID',
                 ]]


In [7]:
df = df[~(df['EVENT_ID'].str.contains('REG', case=False))]
df = df[~(df['EVENT_ID'].str.contains('STDY', case=False))]


In [8]:
print(df.shape)
df.head()

(5092, 11)


,ACADEMIC_YEAR,ACADEMIC_TERM,ACADEMIC_SESSION,EVENT_ID,EVENT_SUB_TYPE,SECTION,DAY,START_TIME,END_TIME,BUILDING_CODE,ROOM_ID
0,2011,SPRING,MAIN,ACC 101,LEC,01,MWF,1900-01-01 09:05:00,1900-01-01 10:00:00,PICKTT,223
1,2011,SPRING,MAIN,ACC 101,LEC,02,MWF,1900-01-01 11:15:00,1900-01-01 12:10:00,PICKTT,223
2,2011,SPRING,MAIN,ACC 102,LEC,01,TTHR,1900-01-01 11:10:00,1900-01-01 12:35:00,PICKTT,223
3,2011,SPRING,MAIN,BIO 102,LEC,01,MWF,1900-01-01 09:05:00,1900-01-01 10:00:00,FREER,AUD
4,2011,SPRING,MAIN,BIO 102,LAB,01,MON,1900-01-01 14:30:00,1900-01-01 17:30:00,FREER,117


In [9]:
df.loc[:, 'section_integration_id'] = (df['EVENT_ID'] + '.' +
                                       df['EVENT_SUB_TYPE'] + '.' +
                                       df['ACADEMIC_YEAR'] + '.' +
                                       df['ACADEMIC_TERM'].str.title() + '.' +
                                       df['SECTION']
                                      )


In [ ]:
df = df[~df['EVENT_SUB_TYPE'].isnull()]

In [13]:
df['DAY'].value_counts()

TTHR    1472
MWF      896
MW       605
MON      378
WED      369
TUE      345
FRI      284
THUR     266
ALL      115
TBD       98
ONLN      75
MTWR      45
SAT       34
MF        30
SUN       22
MTU       12
WF        11
WTHR      11
TWTF       7
CANC       7
MTRF       4
MTWF       2
MTW        2
THUF       1
WTF        1
Name: DAY, dtype: int64

In [12]:
df['EVENT_SUB_TYPE'].value_counts()

LEC     2808
COMB    1081
LAB      979
ACE       73
EXT       57
PRAC      48
SI        30
HYBD      12
ONLN       4
Name: EVENT_SUB_TYPE, dtype: int64

In [11]:
df['BUILDING_CODE'].value_counts()

FREER     1566
PICKTT    1230
CANTWL    1074
JWAL       379
           260
SARATG     193
TOMPKI     187
JWSC        60
ONLINE      36
PAOLZI      34
SAUN        22
FORCC       20
UPR          8
WOOD         7
VIC          5
SUGARB       4
OVL          3
WELD         2
LAMBRT       2
Name: BUILDING_CODE, dtype: int64

In [ ]:
df = df.rename(columns={
                        'START_TIME': 'start_time',
                        'END_TIME': 'end_time',
                        'BUILDING_CODE': 'building',
                        'ROOM_ID': 'room',
                       })


In [ ]:
df = df.loc[:, ['section_integration_id', 'meeting_days',
                'start_time', 'end_time',
                'building', 'room', 
               ]]


In [ ]:
df = (df.sort_values(['section_integration_id', 
                      'meeting_days', 'meeting_time'])
        .drop_duplicates(['section_integration_id', 
                          'meeting_days', 'meeting_time'],
                         keep='last')
     )


In [ ]:
print(df.shape)

In [ ]:
fn_output = f'{today_str}_section_schedules.txt'
df.to_csv(fn_output, index=False)
